# CONTENT BASED FILTERING
Content based filtering berdasarkan pada data aktivitas, kategori, serta rating destinasi wisata. Sistem ini merekomendasikan destinasi wisata berdasarkan destinasi yang pernah dikunjungi atau diklik pengguna. Pengguna diberikan rekomendasi dengan aktivitas yang mirip dengan destinasi wisata sebelumnya.

In [55]:
import json
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

**1. Memuat data dari file JSON**

In [56]:
# URL ke data JSON
url = 'https://raw.githubusercontent.com/acoramadan/Paradisata/CC/Database/Tourism_place/places_by_city.json'

# Memuat data JSON dari URL
response = requests.get(url)

# Mengubah JSON menjadi DataFrame
data = pd.DataFrame(places_data)


 **2. Memproses fitur**


In [57]:
# Mengekstrak fitur kategori dan aktivitas
data['kategori'] = data['kategori'].astype(str)
data['aktivitas'] = data['aktivitas'].apply(lambda x: ' '.join(x))

# Menggunakan TF-IDF untuk kategori dan aktivitas
# TF-IDF mengubah data teks menjadi representasi numerik
vectorizer = TfidfVectorizer()
kategori_tfidf = vectorizer.fit_transform(data['kategori']).toarray()
aktivitas_tfidf = vectorizer.fit_transform(data['aktivitas']).toarray()

# Normalisasi rating agar memudahkan kinerja machine learning
scaler = MinMaxScaler()
rating_scaled = scaler.fit_transform(data[['rating']])

# Menggabungkan semua fitur
features_combined = np.hstack([kategori_tfidf, aktivitas_tfidf, rating_scaled])

4. Definisi Model

In [58]:
# Membuat model Neural Network
model = tf.keras.Sequential([
    layers.Input(shape=(features_combined.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(features_combined.shape[0], activation='softmax')  # Output sesuai jumlah destinasi
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Labels untuk training (one-hot encoding)
labels = np.eye(features_combined.shape[0])

# Training model
history=model.fit(features_combined, labels, epochs=50, batch_size=8, verbose=1)

# Evaluasi model
loss, accuracy = model.evaluate(features_combined, labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Simpan model TensorFlow
model.save('content_based_model.h5')

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 4.9675
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0520 - loss: 4.9224     
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1084 - loss: 4.8754  
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1520 - loss: 4.8100 
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2101 - loss: 4.7007 
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2022 - loss: 4.5632
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1921 - loss: 4.3148
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1320 - loss: 4.1097     
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2961 - loss: 3.7053 
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2822 - loss: 3.4352 
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4289 - loss: 3.1601  
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

Accuracy: 94.41%


**5. Konversi ke TFLite (Setelah model selesai dilatih dan diuji)**

In [61]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('content_based_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model berhasil dikonversi ke TFLite!")

Saved artifact at '/tmp/tmpokk_nhrw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 218), dtype=tf.float32, name='keras_tensor_40')
Output Type:
  TensorSpec(shape=(None, 143), dtype=tf.float32, name=None)
Captures:
  138202634974864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138202633509824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138202633512640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138202633511760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138202633507888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138202633514576: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model berhasil dikonversi ke TFLite!


**6. Pengujian Model (menggunakan TFLite Interpreter)**

In [62]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Mendapatkan tensor input dan output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Menguji rekomendasi dengan input tertentu
test_input = features_combined[5].reshape(1, -1)  # Ambil data pertama sebagai contoh
input_data = np.array(test_input, dtype=np.float32)

# Memberikan input ke model
interpreter.set_tensor(input_details[0]['index'], input_data)

# Menjalankan inferensi
interpreter.invoke()

# Mengambil hasil output
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_class = np.argmax(output_data)
print(f"Prediksi destinasi wisata dengan ID {predicted_class} (indeks) dengan probabilitas {output_data[0][predicted_class]}")

Prediksi destinasi wisata dengan ID 5 (indeks) dengan probabilitas 0.9783647656440735


**7. Hasil rekomendasi tempat**

In [63]:
input_data = features_combined[5].reshape(1, -1)
predictions = model.predict(input_data)
names = data['nama'].tolist()

# Top 20 rekomendasi
top_indices = np.argsort(predictions[0])[-20:][::-1]
print("\nRekomendasi Destinasi:")
for idx in top_indices:
    print(f"{names[idx]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step

Rekomendasi Destinasi:
Gedung Sate
Candi Cangkuang
Rumah Sejarah Kalijati
Benteng Pendem
Kampung Cina
Candi Jiwa
Museum Geologi
Benteng Karawang
Pantai Ujung Genteng
Kebun Teh Purwakarta
Pantai Cikao
Kota Tua Bogor
Cikao Park
Keraton Kasepuhan
Benteng Cagar Alam Pangandaran
Istana Bogor
Curug Cimahi
Tugu Perjuangan Karawang
Situ Gunung
Kawah Cibuni
